In [47]:
import numpy as np
import pandas as pd
import re
from pathlib import Path

In [48]:
df = pd.read_excel('./raw_data/SPX_aug_sept.xlsx')

In [49]:
df

,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume
0,Summary,6448.26,210.25,6271.71,6508.23,6271.71,541200.0,N.A.
1,04AUG2025_00:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:30 - 09:31,6279.90,41.89,6271.71,6280.73,6271.71,60.0,N.A.
3,09:31 - 09:32,6283.52,3.62,6280.50,6283.52,6278.63,60.0,N.A.
4,09:32 - 09:33,6284.42,0.90,6283.49,6285.22,6283.41,60.0,N.A.
...,...,...,...,...,...,...,...,...
9038,16:15 - 16:16,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9039,16:16 - 16:17,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9040,16:17 - 16:18,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9041,16:18 - 16:19,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.


In [50]:
def add_datetime_column(df):
    """
    Add DateTime column to DataFrame with date headers and time intervals.
    
    Detects date header rows (e.g., '04AUG2025'), propagates dates to subsequent rows,
    and combines with time intervals to create datetime objects in US/Eastern timezone.
    
    Args:
        df: DataFrame with 'Time Interval' column and date header rows
        
    Returns:
        DataFrame with 'DateTime' column added (date headers removed)
    """
    # Initialize variables
    current_date = None
    rows_to_keep = []
    
    # Process each row
    for idx, row in df.iterrows():
        time_interval = str(row.get('Time Interval', ''))
        
        # Check if this is a date header row (pattern: 04AUG2025)
        date_pattern = r'\d{1,2}[A-Za-z]{3}\d{4}'
        match = re.search(date_pattern, time_interval)
        
        if match:
            # Parse the date
            date_str = match.group().upper()
            current_date = pd.to_datetime(date_str, format='%d%b%Y').date()
        elif current_date is not None:
            # This is a data row - keep it with the current date
            row_dict = row.to_dict()
            row_dict['_date'] = current_date
            rows_to_keep.append(row_dict)
    
    # Create new DataFrame
    df_result = pd.DataFrame(rows_to_keep)
    
    # Extract start time from interval and combine with date
    def parse_datetime(row):
        time_str = str(row['Time Interval']).strip()
        
        # Extract start time (e.g., "09:30" from "09:30 - 09:31")
        if ' - ' in time_str:
            start_time = time_str.split(' - ')[0].strip()
        elif '-' in time_str:
            start_time = time_str.split('-')[0].strip()
        else:
            start_time = time_str
        
        # Combine date and time
        datetime_str = f"{row['_date']} {start_time}"
        dt = pd.to_datetime(datetime_str)
        
        # Localize to US/Eastern
        dt = dt.tz_localize('US/Eastern')
        
        return dt
    
    df_result['DateTime'] = df_result.apply(parse_datetime, axis=1)
    
    # Drop temporary date column
    df_result = df_result.drop('_date', axis=1)
    
    # Reorder columns (DateTime first)
    cols = ['DateTime'] + [col for col in df_result.columns if col != 'DateTime']
    df_result = df_result[cols]
    
    return df_result

In [45]:
add_datetime_column(df)

,DateTime,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume
0,2025-08-04 09:30:00-04:00,09:30 - 09:31,6279.90,41.89,6271.71,6280.73,6271.71,60.0,N.A.
1,2025-08-04 09:31:00-04:00,09:31 - 09:32,6283.52,3.62,6280.50,6283.52,6278.63,60.0,N.A.
2,2025-08-04 09:32:00-04:00,09:32 - 09:33,6284.42,0.90,6283.49,6285.22,6283.41,60.0,N.A.
3,2025-08-04 09:33:00-04:00,09:33 - 09:34,6287.32,2.90,6284.57,6289.00,6284.57,60.0,N.A.
4,2025-08-04 09:34:00-04:00,09:34 - 09:35,6288.58,1.26,6286.90,6288.58,6285.79,60.0,N.A.
...,...,...,...,...,...,...,...,...,...
9015,2025-09-03 16:15:00-04:00,16:15 - 16:16,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9016,2025-09-03 16:16:00-04:00,16:16 - 16:17,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9017,2025-09-03 16:17:00-04:00,16:17 - 16:18,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.
9018,2025-09-03 16:18:00-04:00,16:18 - 16:19,6448.26,0.00,6448.26,6448.26,6448.26,60.0,N.A.


In [51]:
def load_and_consolidate_data(raw_data_dir='raw_data'):
    raw_data_path = Path(raw_data_dir)
    
    excel_files = list(raw_data_path.glob('*.xlsx'))
    
    tickers = set()
    for file in excel_files:
        ticker = file.stem.split('_')[0]
        tickers.add(ticker)
    
    ticker_dfs = {ticker: [] for ticker in tickers}
    
    for file in excel_files:
        ticker = file.stem.split('_')[0]
        
        df = pd.read_excel(file)
        
        df = add_datetime_column(df)
        
        ticker_dfs[ticker].append(df)
    
    consolidated_dfs = {}
    for ticker, df_list in ticker_dfs.items():
        consolidated_dfs[ticker] = pd.concat(df_list, ignore_index=True)
        if 'datetime' in consolidated_dfs[ticker].columns:
            consolidated_dfs[ticker] = consolidated_dfs[ticker].sort_values('datetime').reset_index(drop=True)
        consolidated_dfs[ticker] = consolidated_dfs[ticker].drop_duplicates(subset=['datetime'])
    
    return consolidated_dfs

In [39]:

ticker_data = load_and_consolidate_data()


In [46]:
ticker_data['GLDM']

,DateTime,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume
0,2025-10-06 09:30:00-04:00,09:30 - 09:31,77.835,0.915,77.8700,77.8993,77.8049,74.0,120382.0
1,2025-10-06 09:31:00-04:00,09:31 - 09:32,77.800,-0.035,77.8400,77.8400,77.7950,87.0,14869.0
2,2025-10-06 09:32:00-04:00,09:32 - 09:33,77.790,-0.010,77.8051,77.8276,77.7800,99.0,15844.0
3,2025-10-06 09:33:00-04:00,09:33 - 09:34,77.810,0.020,77.7935,77.8276,77.7900,107.0,26003.0
4,2025-10-06 09:34:00-04:00,09:34 - 09:35,77.780,-0.030,77.8100,77.8100,77.7650,58.0,14330.0
...,...,...,...,...,...,...,...,...,...
33927,2025-12-03 15:55:00-05:00,15:55 - 15:56,83.250,-0.020,83.2650,83.2700,83.2500,46.0,21170
33928,2025-12-03 15:56:00-05:00,15:56 - 15:57,83.210,-0.040,83.2565,83.2565,83.2100,23.0,9776
33929,2025-12-03 15:57:00-05:00,15:57 - 15:58,83.255,0.045,83.2100,83.2550,83.2100,16.0,7458
33930,2025-12-03 15:58:00-05:00,15:58 - 15:59,83.270,0.015,83.2500,83.2700,83.2500,24.0,15261
